In [20]:
import os 
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults 
from langchain.agents import tool
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
from PIL import Image
from IPython.display import display



In [5]:
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [32]:
tavily_tool = TavilySearchResults(api_key =TAVILY_API_KEY)

@tool

def research_agent(input_query: str) -> str:
    """Uses Tavily To get the search results for a query ."""
    search_results =tavily_tool.run(input_query,num_results=5)
    print(f"Reseach Results : {search_results}")
    return search_results


In [33]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY,temperature=0.7)

def answer_drafter(research_output: str) -> str:
    print(f"Drafting the answer ...")
    prompt = f""" you are an expert researcher .
    use the following search results to answer the question in a clear and detailed way .

    Search Results:
    {research_output}

Write a well structured Answer:

""" 
    Drafted_answer=llm.invoke(prompt).content 
    print(f"Drafted_answer: {Drafted_answer}")
    return Drafted_answer



In [34]:
# Defining the Langgraph work Flow
workflow = StateGraph(input="input_query", output="final_answer")


# adding the two agents workflow 
workflow.add_node("research", RunnableLambda(research_agent))
workflow.add_node("draft",RunnableLambda(answer_drafter))

# seting  up the flow 

workflow.set_entry_point("research")
workflow.add_edge("research","draft")
workflow.set_finish_point("draft")


# compile the graph 

graph_app = workflow.compile()


C:\Users\91832\AppData\Local\Temp\ipykernel_5300\543328916.py:2: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  workflow = StateGraph(input="input_query", output="final_answer")
c:\Users\91832\Desktop\Tavily_web\tavily\lib\site-packages\langgraph\graph\state.py:89: UserWarning: Invalid state_schema: input_query. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(
c:\Users\91832\Desktop\Tavily_web\tavily\lib\site-packages\langgraph\graph\state.py:89: UserWarning: Invalid state_schema: final_answer. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(


In [ ]:
query = "what is langchain and what are the importance of it and how it is different to others  ?"

results = graph_app.invoke({'input_query':query})

#print the final results

print("Final Answer :")

print(results)

Reseach Results : [{'title': 'What is langchain? - Reddit', 'url': 'https://www.reddit.com/r/LangChain/comments/13895ru/what_is_langchain/', 'content': 'LangChain. LangChain is an open-source framework and developer toolkit that helps developers get LLM applications from prototype to production.', 'score': 0.9341134}, {'title': 'What Is LangChain? - IBM', 'url': 'https://www.ibm.com/think/topics/langchain', 'content': 'What Is LangChain? | IBM\nMy IBM Log in Subscribe   \nWhat is LangChain?\nArtificial Intelligence\n31 October 2023\nLink copied\nWhat is LangChain?\nLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain’s tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents. [...] LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts neces